# Data Collections

## Code list

1. [Modules, Tools import](#1-modules-tools-import)
2. [Data import & Setup](#2-data-import--setup)
3. [Data Merge](#3-data-merge)
   1. [GDP Merge](#1gdp-merge)
   2. [Interest rate Merge](#2-interest-rate-merge)
   3. [School district Merge](#3-school-district-merge)
   4. [Lease rate Merge](#4-lease-rate-merge)
   5. [Number of Redevelopment Merge](#5-number-of-redevelopment-merge)
   6. [Near the forest imformation Merge](#6-near-the-forest-imformation-merge)
   7. [Near the river imformation Merge](#7-near-the-river-imformation-merge)
   8. [Near the park imformation Merge](#8-near-the-park-imformation-merge)
   9. [Near the working road imformation Merge](#9-near-the-working-road-imformation-merge)
   10. [Near the ecological park imformation Merge](#10-near-the-ecological-park-imformation-merge)
4.  [Data Save](#4-data-save)

## 1. Modules, Tools import

In [126]:
# utils
import pandas as pd
import numpy as np

## 2. Data import & Setup

In [127]:
# 필요한 data load
train_path = '../../data/train.csv'
test_path  = '../../data/test.csv'
codebook_path = '../../data/codebook.csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
codebook = pd.read_csv(codebook_path)

/var/folders/kv/46dvhwp932v4b692v761hjdc0000gn/T/ipykernel_60073/3015333637.py:5: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(train_path)


In [128]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
train['is_test'] = 0
test['is_test'] = 1
concat = pd.concat([train, test])     # 하나의 데이터로 만들어줍니다.

In [129]:
concat.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target', 'is_test'],
      dtype='object')

In [130]:
concat['계약년'] = concat['계약년월'].astype('str').map(lambda x : x[:4])
concat['계약월'] = concat['계약년월'].astype('str').map(lambda x : x[4:])

In [131]:
concat['구'] = concat['시군구'].map(lambda x : x.split()[1])
concat['동'] = concat['시군구'].map(lambda x : x.split()[2])

## 3. Data merge

### 1.GDP Merge

In [132]:
gdp_path = '../../data/gdp(2007~).csv'
gdp = pd.read_csv(gdp_path)

In [133]:
# GDP 데이터는 분기마다 존재하므로 Merge의 키로 사용하기 위해 생성
# 월에 따른 분기를 만드는 함수
def make_quarter(month):
    if month in ["01", "02", "03"]:
        return "1/4"
    elif month in ["04", "05", "06"]:
        return "2/4"
    elif month in ["07", "08", "09"]:
        return "3/4"
    elif month in ["10", "11", "12"]:
        return "4/4"
    else:
        return "NaN" 

In [134]:
concat["계약분기"] = concat["계약월"].apply(make_quarter)

In [135]:
# 계약년에 분기를 더해 분기(계약년) 생성
concat["분기"] = concat['계약년']+concat["계약분기"]

In [136]:
# 분기를 기준으로 병합 1128094 rows 
new_concat = pd.merge(concat, gdp, on="분기", how="inner")
new_concat.shape

(1128094, 61)

In [137]:
# 병합하기 전 원본 1128094 rows
concat.shape

(1128094, 59)

In [138]:
# 잘못된 병합으로 생성된 NaN 값 조회 -> 데이터 소실 없이 정상적으로 Merge
new_concat[["국내총생산(명목GDP)", "경제성장률(실질GDP성장률)"]].isna().sum()

국내총생산(명목GDP)       0
경제성장률(실질GDP성장률)    0
dtype: int64

In [139]:
# 병합에 사용한 columns 삭제
del new_concat["분기"]
del new_concat["계약분기"]

### 2. Interest rate merge

In [140]:
interest_rate_path = '../../data/ interest_rate.csv'
interest_rate = pd.read_csv(interest_rate_path)

In [141]:
interest_rate

,변경일자_년,변경일자_월일,기준금리
0,2023,01월 13일,3.50
1,2022,11월 24일,3.25
2,2022,10월 12일,3.00
3,2022,08월 25일,2.50
4,2022,07월 13일,2.25
5,2022,05월 26일,1.75
6,2022,04월 14일,1.50
7,2022,01월 14일,1.25
8,2021,11월 25일,1.00
9,2021,08월 26일,0.75


In [142]:
# 변경일자년월일 생성 -> Merge 키로 사용
interest_rate["일자년월일"] = interest_rate["변경일자_년"].astype(str) + interest_rate["변경일자_월일"].map(lambda x : x[:2] + x[4:6])


In [143]:
use_interest_rate = interest_rate.sort_values("일자년월일")[["일자년월일", "기준금리"]]

In [144]:
use_interest_rate.head(10)

,일자년월일,기준금리
38,20060810,4.50
37,20070712,4.75
36,20070809,5.00
35,20080807,5.25
34,20081009,5.00
33,20081027,4.25
32,20081107,4.00
31,20081211,3.00
30,20090109,2.50
29,20090212,2.00


In [145]:
# 변경일자년월일 생성 -> Merge 키로 사용, use_interest_rate와 같은 형태로 생성
new_concat["일자년월일"] = new_concat["계약년월"].astype(str)+new_concat["계약일"].astype(str).str.zfill(2)

In [146]:
new_concat.sort_values(by="일자년월일", inplace=True)

In [147]:
new_concat["일자년월일"].head(10)

1079005    20070101
1082208    20070101
1083444    20070101
1090106    20070101
1082788    20070101
1086015    20070101
1077899    20070101
1090026    20070101
1077914    20070101
1087006    20070101
Name: 일자년월일, dtype: object

In [148]:
new_concat = pd.merge(new_concat, use_interest_rate, on="일자년월일", how='left')

In [149]:
# 데이터에 들어간 값 확인
new_concat[~new_concat["기준금리"].isna()].sort_values(by="일자년월일")

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,target,is_test,계약년,계약월,구,동,국내총생산(명목GDP),경제성장률(실질GDP성장률),일자년월일,기준금리
28839,서울특별시 성북구 정릉동,239,239.0,0.0,정릉풍림아이원,114.750,200707,12,13,2005,...,46400.0,0,2007,07,성북구,정릉동,"275,865.80",5.5,20070712,4.75
28954,서울특별시 중구 회현동1가,100-12,100.0,12.0,이지-빌,55.610,200707,12,2,2006,...,19200.0,0,2007,07,중구,회현동1가,"275,865.80",5.5,20070712,4.75
28955,서울특별시 노원구 하계동,273,273.0,0.0,장미,43.560,200707,12,3,1989,...,11500.0,0,2007,07,노원구,하계동,"275,865.80",5.5,20070712,4.75
28956,서울특별시 도봉구 창동,347,347.0,0.0,창동주공3단지,66.560,200707,12,3,1990,...,29000.0,0,2007,07,도봉구,창동,"275,865.80",5.5,20070712,4.75
28957,서울특별시 동대문구 회기동,347-7,347.0,7.0,회기,81.860,200707,12,4,2003,...,22000.0,0,2007,07,동대문구,회기동,"275,865.80",5.5,20070712,4.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101759,서울특별시 도봉구 방학동,720-18,720.0,18.0,방학동삼성래미안1,118.381,202301,13,8,2002,...,76000.0,0,2023,01,도봉구,방학동,"523,816.20",0.9,20230113,3.50
1101758,서울특별시 구로구 구로동,685-222,685.0,222.0,주공1,73.080,202301,13,9,1986,...,71000.0,0,2023,01,구로구,구로동,"523,816.20",0.9,20230113,3.50
1101757,서울특별시 노원구 상계동,995,995.0,0.0,한신4차,84.840,202301,13,12,1995,...,52000.0,0,2023,01,노원구,상계동,"523,816.20",0.9,20230113,3.50
1101765,서울특별시 성북구 장위동,65-239,65.0,239.0,라인캐슬,55.150,202301,13,5,2022,...,62450.0,0,2023,01,성북구,장위동,"523,816.20",0.9,20230113,3.50


In [150]:
# 새로운 데이터가 나올 때까지 NaN 값을 이전 금리로 채움
new_concat["기준금리"].fillna(method='ffill', inplace=True)

/var/folders/kv/46dvhwp932v4b692v761hjdc0000gn/T/ipykernel_60073/572768861.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_concat["기준금리"].fillna(method='ffill', inplace=True)


In [151]:
# 기준금리는 2007년 7월 12일 기준으로 바뀌었으니 이전 기준금리는 Merge해도 NaN 값임
new_concat[new_concat["기준금리"].isna()].sort_values(by="일자년월일")

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,target,is_test,계약년,계약월,구,동,국내총생산(명목GDP),경제성장률(실질GDP성장률),일자년월일,기준금리
0,서울특별시 강서구 마곡동,429,429.0,0.0,벽산,59.980,200701,1,15,1999,...,30000.0,0,2007,01,강서구,마곡동,"252,586.30",4.9,20070101,NaN
13,서울특별시 성북구 하월곡동,225,225.0,0.0,래미안월곡,114.678,200701,1,18,2006,...,70000.0,0,2007,01,성북구,하월곡동,"252,586.30",4.9,20070101,NaN
12,서울특별시 동작구 신대방동,395-62,395.0,62.0,삼성보라매옴니타워,164.760,200701,1,15,1997,...,77000.0,0,2007,01,동작구,신대방동,"252,586.30",4.9,20070101,NaN
10,서울특별시 동대문구 답십리동,42,42.0,0.0,동답한신,59.220,200701,1,7,1992,...,18400.0,0,2007,01,동대문구,답십리동,"252,586.30",4.9,20070101,NaN
9,서울특별시 성북구 정릉동,1020,1020.0,0.0,정릉우성,84.990,200701,1,10,2000,...,31500.0,0,2007,01,성북구,정릉동,"252,586.30",4.9,20070101,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28715,서울특별시 성동구 응봉동,98,98.0,0.0,금호현대,79.800,200707,11,9,1990,...,51000.0,0,2007,07,성동구,응봉동,"275,865.80",5.5,20070711,NaN
28716,서울특별시 도봉구 창동,807,807.0,0.0,쌍용,135.000,200707,11,13,1997,...,71900.0,0,2007,07,도봉구,창동,"275,865.80",5.5,20070711,NaN
28717,서울특별시 노원구 상계동,1255,1255.0,0.0,은빛1단지,59.920,200707,11,11,1998,...,20450.0,0,2007,07,노원구,상계동,"275,865.80",5.5,20070711,NaN
28708,서울특별시 노원구 공릉동,90,90.0,0.0,청솔,84.900,200707,11,6,2000,...,39300.0,0,2007,07,노원구,공릉동,"275,865.80",5.5,20070711,NaN


In [152]:
# 따라서 2007년 7월 12일 기준으로 이전 기준금리를 수동으로 채움 (여기서는 2006년 8월 10일 변경된 기준금리인 4.50를 사용)
new_concat["기준금리"].fillna(4.5, inplace=True)

In [153]:
# 사용한 키값 제거
del new_concat["일자년월일"]

In [154]:
new_concat.head(10)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,단지신청일,target,is_test,계약년,계약월,구,동,국내총생산(명목GDP),경제성장률(실질GDP성장률),기준금리
0,서울특별시 강서구 마곡동,429,429.0,0.0,벽산,59.980,200701,1,15,1999,...,NaN,30000.0,0,2007,01,강서구,마곡동,"252,586.30",4.9,4.5
1,서울특별시 노원구 월계동,322-1,322.0,1.0,성원,59.940,200701,1,9,1995,...,2013-03-07 09:46:23.0,15800.0,0,2007,01,노원구,월계동,"252,586.30",4.9,4.5
2,서울특별시 도봉구 도봉동,647,647.0,0.0,도봉파크빌2,84.354,200701,1,1,2002,...,NaN,23500.0,0,2007,01,도봉구,도봉동,"252,586.30",4.9,4.5
3,서울특별시 중랑구 상봉동,495,495.0,0.0,동부,84.990,200701,1,2,1999,...,2013-03-07 09:46:28.0,26900.0,0,2007,01,중랑구,상봉동,"252,586.30",4.9,4.5
4,서울특별시 노원구 중계동,445,445.0,0.0,염광,59.340,200701,1,8,1996,...,NaN,22000.0,0,2007,01,노원구,중계동,"252,586.30",4.9,4.5
5,서울특별시 서대문구 북가좌동,325-18,325.0,18.0,충우팰리스1,84.880,200701,1,3,2004,...,NaN,24000.0,0,2007,01,서대문구,북가좌동,"252,586.30",4.9,4.5
6,서울특별시 강동구 길동,45,45.0,0.0,우림루미아트,84.570,200701,1,7,2003,...,NaN,28000.0,0,2007,01,강동구,길동,"252,586.30",4.9,4.5
7,서울특별시 중랑구 면목동,193-1,193.0,1.0,면목한신,45.440,200701,1,5,1987,...,NaN,11500.0,0,2007,01,중랑구,면목동,"252,586.30",4.9,4.5
8,서울특별시 강동구 길동,456-2,456.0,2.0,하이브,59.970,200701,1,8,2006,...,NaN,21100.0,0,2007,01,강동구,길동,"252,586.30",4.9,4.5
9,서울특별시 성북구 정릉동,1020,1020.0,0.0,정릉우성,84.990,200701,1,10,2000,...,2013-03-07 09:46:31.0,31500.0,0,2007,01,성북구,정릉동,"252,586.30",4.9,4.5


In [155]:
# 데이터 손상 확인
concat.shape

(1128094, 59)

In [156]:
new_concat.shape

(1128094, 60)

In [157]:
## 중간 데이터셋 저장, Window와 Mac을 고려해 cp949, UTF-8 인코딩 파일을 따로 생성
# save_train = new_concat[new_concat["is_test"]==0]
# del save_train["is_test"]
# save_train.to_csv('train_merge_0119_01(cp949).csv', index=False, encoding='cp949')
# save_train.to_csv('train_merge_0119_01(UTF-8).csv', index=False, encoding='UTF-8')


In [158]:
# save_test = new_concat[new_concat["is_test"]==1]
# del save_test["is_test"]
# save_test.to_csv('test_merge_0119_01(cp949).csv', index=False, encoding='cp949')
# save_test.to_csv('test_merge_0119_01(UTF-8).csv', index=False, encoding='UTF-8')

### 3. School district Merge

In [159]:
## 중간 저장시 데이터 업데이트가 필요
## 데이터 업데이트
# train_path = '../../data/train_merge_0119_01(UTF-8).csv'
# test_path  = '../../data/test_merge_0119_01(UTF-8).csv'
# train = pd.read_csv(train_path)
# test = pd.read_csv(test_path)

# # train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
# train['is_test'] = 0
# test['is_test'] = 1
# new_concat = pd.concat([train, test])     # 하나의 데이터로 만들어줍니다.

In [160]:
edu_group_path = '../../data/education_group.csv'
edu_group = pd.read_csv(edu_group_path)

In [161]:
edu_group

,구,학군
0,동대문구,1
1,중랑구,1
2,마포구,2
3,서대문구,2
4,은평구,2
5,구로구,3
6,금천구,3
7,영등포구,3
8,노원구,4
9,도봉구,4


In [162]:
new_concat = pd.merge(new_concat, edu_group, on="구", how='inner')

In [163]:
new_concat[["구", "학군"]]

,구,학군
0,강서구,7
1,강서구,7
2,강서구,7
3,강서구,7
4,강서구,7
...,...,...
1128089,광진구,10
1128090,광진구,10
1128091,광진구,10
1128092,광진구,10


In [164]:
new_concat.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target', 'is_test', '계약년', '계약월', '구', '동',
       '국내총생산(명목GDP)', '경제성장률(실질GDP성장률)', '기준금리', '학군'],
      dtype='object')

In [165]:
concat.shape

(1128094, 59)

In [166]:
new_concat.shape

(1128094, 61)

### 4. Lease rate Merge

In [167]:
lease_rate_path = '../../data/lease_rate.csv'
lease_rate = pd.read_csv(lease_rate_path, encoding='cp949')

In [168]:
# column 명이 금리로 잘못 설정 -> column명을 전세가율로 변경
lease_rate.rename(columns={"금리":"전세가율"}, inplace=True)

In [169]:
lease_rate

,계약년월,전세가율,구
0,201304,50.14,강남구
1,201305,50.17,강남구
2,201306,50.61,강남구
3,201307,51.41,강남구
4,201308,52.07,강남구
...,...,...,...
3145,202305,58.39,중랑구
3146,202306,58.59,중랑구
3147,202307,58.38,중랑구
3148,202308,58.48,중랑구


In [170]:
new_concat.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target', 'is_test', '계약년', '계약월', '구', '동',
       '국내총생산(명목GDP)', '경제성장률(실질GDP성장률)', '기준금리', '학군'],
      dtype='object')

In [171]:
new_concat = pd.merge(new_concat, lease_rate, on=['구', '계약년월'], how='left')

In [172]:
# 전세가율은 2013년도 이후부터 데이터가 존재하므로 이전 데이터는 NaN 값
new_concat

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,is_test,계약년,계약월,구,동,국내총생산(명목GDP),경제성장률(실질GDP성장률),기준금리,학군,전세가율
0,서울특별시 강서구 마곡동,429,429.0,0.0,벽산,59.98,200701,1,15,1999,...,0,2007,01,강서구,마곡동,"252,586.30",4.9,4.5,7,NaN
1,서울특별시 강서구 방화동,832,832.0,0.0,청솔,49.20,200701,2,2,1994,...,0,2007,01,강서구,방화동,"252,586.30",4.9,4.5,7,NaN
2,서울특별시 강서구 방화동,814,814.0,0.0,방화5,33.18,200701,2,13,1994,...,0,2007,01,강서구,방화동,"252,586.30",4.9,4.5,7,NaN
3,서울특별시 강서구 화곡동,1139,1139.0,0.0,초록,59.34,200701,2,10,1998,...,0,2007,01,강서구,화곡동,"252,586.30",4.9,4.5,7,NaN
4,서울특별시 강서구 화곡동,927-1,927.0,1.0,웰라이빌,84.98,200701,2,7,2000,...,0,2007,01,강서구,화곡동,"252,586.30",4.9,4.5,7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 광진구 구의동,631-1,631.0,1.0,현대프라임,59.82,202309,14,21,1997,...,1,2023,09,광진구,구의동,"563,921.50",1.4,3.5,10,51.2
1128090,서울특별시 광진구 자양동,636-11,636.0,11.0,그랜드,48.28,202309,15,6,2023,...,1,2023,09,광진구,자양동,"563,921.50",1.4,3.5,10,51.2
1128091,서울특별시 광진구 광장동,484,484.0,0.0,현대3,59.67,202309,15,8,1990,...,1,2023,09,광진구,광장동,"563,921.50",1.4,3.5,10,51.2
1128092,서울특별시 광진구 구의동,199-18,199.0,18.0,삼성쉐르빌,84.94,202309,16,17,2001,...,1,2023,09,광진구,구의동,"563,921.50",1.4,3.5,10,51.2


In [173]:
concat.shape

(1128094, 59)

In [174]:
new_concat.shape

(1128094, 62)

### 5. Number of Redevelopment Merge

In [175]:
redevelopment_path = '../../data/redevelopment_num.csv'
redevelopment = pd.read_csv(redevelopment_path)

In [176]:
redevelopment.rename(columns={"연도" : "계약년"}, inplace=True)

In [177]:
redevelopment

,구,계약년,재개발개수
0,강남구,2007,0
1,강남구,2008,0
2,강남구,2009,0
3,강남구,2010,0
4,강남구,2011,0
...,...,...,...
420,관악구,2023,0
421,서초구,2023,0
422,강남구,2023,0
423,송파구,2023,0


In [180]:
new_concat["계약년"].dtype

dtype('O')

In [181]:
redevelopment["계약년"].dtype

dtype('int64')

In [183]:
redevelopment["계약년"] = redevelopment["계약년"].astype(str)
redevelopment["계약년"].dtype

dtype('O')

In [184]:
new_concat = pd.merge(new_concat, redevelopment, on=['구', '계약년'], how='left')

In [185]:
new_concat.head(10)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,계약년,계약월,구,동,국내총생산(명목GDP),경제성장률(실질GDP성장률),기준금리,학군,전세가율,재개발개수
0,서울특별시 강서구 마곡동,429,429.0,0.0,벽산,59.98,200701,1,15,1999,...,2007,01,강서구,마곡동,"252,586.30",4.9,4.5,7,NaN,0
1,서울특별시 강서구 방화동,832,832.0,0.0,청솔,49.20,200701,2,2,1994,...,2007,01,강서구,방화동,"252,586.30",4.9,4.5,7,NaN,0
2,서울특별시 강서구 방화동,814,814.0,0.0,방화5,33.18,200701,2,13,1994,...,2007,01,강서구,방화동,"252,586.30",4.9,4.5,7,NaN,0
3,서울특별시 강서구 화곡동,1139,1139.0,0.0,초록,59.34,200701,2,10,1998,...,2007,01,강서구,화곡동,"252,586.30",4.9,4.5,7,NaN,0
4,서울특별시 강서구 화곡동,927-1,927.0,1.0,웰라이빌,84.98,200701,2,7,2000,...,2007,01,강서구,화곡동,"252,586.30",4.9,4.5,7,NaN,0
5,서울특별시 강서구 화곡동,1152,1152.0,0.0,보람,84.98,200701,2,1,2004,...,2007,01,강서구,화곡동,"252,586.30",4.9,4.5,7,NaN,0
6,서울특별시 강서구 화곡동,927-1,927.0,1.0,웰라이빌,79.87,200701,2,8,2000,...,2007,01,강서구,화곡동,"252,586.30",4.9,4.5,7,NaN,0
7,서울특별시 강서구 내발산동,671-13,671.0,13.0,청솔샤르망1004동,84.60,200701,2,5,2005,...,2007,01,강서구,내발산동,"252,586.30",4.9,4.5,7,NaN,0
8,서울특별시 강서구 등촌동,713,713.0,0.0,대동황토방2,80.93,200701,2,14,2003,...,2007,01,강서구,등촌동,"252,586.30",4.9,4.5,7,NaN,0
9,서울특별시 강서구 내발산동,712-2,712.0,2.0,유니드,73.51,200701,2,4,2005,...,2007,01,강서구,내발산동,"252,586.30",4.9,4.5,7,NaN,0


In [186]:
new_concat["재개발개수"].isna().sum()

0

In [187]:
concat.shape

(1128094, 59)

In [189]:
new_concat.shape

(1128094, 63)

In [185]:
## 병합 데이터 중간 저장
# save_train = new_concat[new_concat["is_test"]==0]
# del save_train["is_test"]
# save_train.to_csv('train_merge_0119_02(cp949).csv', index=False, encoding='cp949')
# save_train.to_csv('train_merge_0119_02(UTF-8).csv', index=False, encoding='UTF-8')

In [191]:
# save_test = new_concat[new_concat["is_test"]==1]
# del save_test["is_test"]
# save_test.to_csv('test_merge_0119_02(cp949).csv', index=False, encoding='cp949')
# save_test.to_csv('test_merge_0119_02(UTF-8).csv', index=False, encoding='UTF-8')

### 6. Near the forest imformation Merge

In [209]:
forest_range_score_path = '../../data/forest_range_score.csv'
index_path = '../../data/forest_range_score_categori.csv'
forest_range_score = pd.read_csv(forest_range_score_path, encoding='UTF-8')
index = pd.read_csv(index_path, encoding='cp949')

In [210]:
# # 데이터 업데이트
# train_path = 'train_merge_0119_02(UTF-8).csv'
# test_path  = 'test_merge_0119_02(UTF-8).csv'
# train = pd.read_csv(train_path)
# test = pd.read_csv(test_path)

# # train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
# train['is_test'] = 0
# test['is_test'] = 1
# concat = pd.concat([train, test])     # 하나의 데이터로 만들어줍니다.

In [211]:
index

,테이블명(영문),테이블명(한글),컬럼명(영문),컬럼명(한글),타입
0,TB_APRTM_FREST_LILHD_NDX,서울시 아파트별 숲세권 생활지수,SGNG_CD,시군구코드,VARCHAR(5)
1,TB_APRTM_FREST_LILHD_NDX,서울시 아파트별 숲세권 생활지수,LGDNG_CD,법정동코드,VARCHAR(10)
2,TB_APRTM_FREST_LILHD_NDX,서울시 아파트별 숲세권 생활지수,HSCMP_CD,단지코드,VARCHAR(9)
3,TB_APRTM_FREST_LILHD_NDX,서울시 아파트별 숲세권 생활지수,HSCMP_NM,단지명,VARCHAR(50)
4,TB_APRTM_FREST_LILHD_NDX,서울시 아파트별 숲세권 생활지수,LGDNG_ADDR,법정동주소,VARCHAR(100)
5,TB_APRTM_FREST_LILHD_NDX,서울시 아파트별 숲세권 생활지수,LILHD_NDX,생활지수,NUMBER(24)
6,TB_APRTM_FREST_LILHD_NDX,서울시 아파트별 숲세권 생활지수,LILHD_NDX_NM,생활지수명,VARCHAR(10)
7,TB_APRTM_FREST_LILHD_NDX,서울시 아파트별 숲세권 생활지수,EVSCR,평가점수,"NUMBER(5,2)"


In [212]:
forest_range_score

,sgng_cd,lgdng_cd,hscmp_cd,hscmp_nm,lgdng_addr,lilhd_ndx,lilhd_ndx_nm,evscr
0,11110,1111011500,A11005401,광화문풍림스페이스본 아파트,서울특별시 종로구 사직동 9 광화문풍림스페이스본 아파트,2,중상위권,76
1,11110,1111011800,A11007001,경희궁의아침3단지,서울특별시 종로구 내수동 72 경희궁의아침3단지,2,중상위권,84
2,11110,1111011800,A11087101,경희궁의아침2단지,서울특별시 종로구 내수동 71 경희궁의아침2단지,2,중상위권,84
3,11110,1111012400,A11014001,로얄팰리스스위트,서울특별시 종로구 수송동 85 로얄팰리스스위트,2,중상위권,77
4,11110,1111013300,A11034001,현대뜨레비앙,서울특별시 종로구 익선동 55 현대뜨레비앙,1,상위권,86
...,...,...,...,...,...,...,...,...
2638,11740,1174011000,A13410006,강일리버파크4단지,서울특별시 강동구 강일동 673 강일리버파크4단지,4,중하위권,63
2639,11740,1174011000,A13410004,강일리버파크6단지,서울특별시 강동구 강일동 684 강일리버파크6단지,3,중위권,73
2640,11740,1174011000,A13410010,강일리버파크7단지,서울특별시 강동구 강일동 686 강일리버파크7단지,3,중위권,64
2641,11740,1174011000,A13410009,강일리버파크5단지,서울특별시 강동구 강일동 670 강일리버파크5단지,4,중하위권,63


In [213]:
index["컬럼명(영문)"] = index["컬럼명(영문)"].str.lower()

In [214]:
variables = index.set_index('컬럼명(영문)').to_dict()['컬럼명(한글)'] # 코드북에 있는 한글 변수명 column을 키로 영문 변수명 column을 value로 한 딕셔너리타입 변수 생성
variables

{'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'lgdng_addr': '법정동주소',
 'lilhd_ndx': '생활지수',
 'lilhd_ndx_nm': '생활지수명',
 'evscr': '평가점수'}

In [215]:
variables = {'sgng_cd': '시군구코드',
             'lgdng_cd': '법정동코드',
             'hscmp_cd': '단지코드',
             'hscmp_nm': '단지명',
             'lgdng_addr': '법정동주소',
             'lilhd_ndx': '숲세권생활지수',
             'lilhd_ndx_nm': '숲세권생활지수명',
             'evscr': '숲세권평가점수'}

In [216]:
variables

{'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'lgdng_addr': '법정동주소',
 'lilhd_ndx': '숲세권생활지수',
 'lilhd_ndx_nm': '숲세권생활지수명',
 'evscr': '숲세권평가점수'}

In [217]:
forest_range_score.rename(columns=variables, inplace=True) # column의 이름 변경

In [218]:
# 필요없는 columns 삭제
use_forest_range_score = forest_range_score.drop(columns=["시군구코드", "법정동코드", "단지코드"], axis=1)

In [219]:
# 뒤의 단지명을 삭제
def remove_substring(df):
    return df["법정동주소"].replace(df["단지명"], "")    

In [220]:
use_forest_range_score["법정동주소"] = use_forest_range_score.apply(remove_substring, axis=1)

In [221]:
# 사용하고 난 단지명 column 삭제
del use_forest_range_score["단지명"]

In [222]:
# 법정동주소의 앞, 뒤 공백 제거 -> 공백으로 인한 Merge 오류 방지
use_forest_range_score["법정동주소"] = use_forest_range_score["법정동주소"].str.strip()

In [223]:
# Merge를 위해 중복된 법정동주소 삭제
use_forest_range_score = use_forest_range_score[~use_forest_range_score["법정동주소"].duplicated()]

In [226]:
# Merge 키인 법정동주소 생성
new_concat["법정동주소"] = new_concat["시군구"] + " " + new_concat["번지"].astype(str)

In [228]:
new_concat = pd.merge(new_concat, use_forest_range_score, on=["법정동주소"], how='left')

In [230]:
concat.shape

(1128094, 60)

In [231]:
new_concat.shape

(1128094, 67)

### 7. Near the river imformation Merge


In [232]:
# 지천 생활권 정보
river_range_path = '../../data/river_stamp.csv'
index_path = '../../data/river_stamp_categori.csv'
river_range = pd.read_csv(river_range_path, encoding='cp949')
index = pd.read_csv(index_path, encoding='cp949')

In [233]:
river_range

,sgng_cd,lgdng_cd,hscmp_cd,hscmp_nm,nmhsh,lgdng_addr,cours_cnt,cours_tot_lngth,lilhd_ndx,lilhd_ndx_nm,evscr
0,11110,1111011500,A11005401,광화문풍림스페이스본 아파트,744,서울특별시 종로구 사직동 9 광화문풍림스페이스본 아파트,0,0.00,5,하위권,0
1,11110,1111011800,A11007001,경희궁의아침3단지,150,서울특별시 종로구 내수동 72 경희궁의아침3단지,1,8179.50,4,중하위권,84
2,11110,1111011800,A11087101,경희궁의아침2단지,90,서울특별시 종로구 내수동 71 경희궁의아침2단지,1,8179.50,4,중하위권,84
3,11110,1111012400,A11014001,로얄팰리스스위트,23,서울특별시 종로구 수송동 85 로얄팰리스스위트,1,8179.50,4,중하위권,84
4,11110,1111013300,A11034001,현대뜨레비앙,291,서울특별시 종로구 익선동 55 현대뜨레비앙,1,8179.50,4,중하위권,84
...,...,...,...,...,...,...,...,...,...,...,...
2638,11740,1174011000,A13410006,강일리버파크4단지,748,서울특별시 강동구 강일동 673 강일리버파크4단지,2,8064.31,4,중하위권,84
2639,11740,1174011000,A13410004,강일리버파크6단지,553,서울특별시 강동구 강일동 684 강일리버파크6단지,2,8064.31,4,중하위권,84
2640,11740,1174011000,A13410010,강일리버파크7단지,731,서울특별시 강동구 강일동 686 강일리버파크7단지,2,8064.31,4,중하위권,84
2641,11740,1174011000,A13410009,강일리버파크5단지,722,서울특별시 강동구 강일동 670 강일리버파크5단지,2,8064.31,4,중하위권,84


In [234]:
index

,테이블명(영문),테이블명(한글),컬럼명(영문),컬럼명(한글),타입
0,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,SGNG_CD,시군구코드,VARCHAR(5)
1,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,LGDNG_CD,법정동코드,VARCHAR(10)
2,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,HSCMP_CD,단지코드,VARCHAR(9)
3,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,HSCMP_NM,단지명,VARCHAR(50)
4,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,NMHSH,세대수,NUMBER(20)
5,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,LGDNG_ADDR,법정동주소,VARCHAR(100)
6,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,COURS_CNT,코스수,NUMBER(14)
7,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,COURS_TOT_LNGTH,코스총길이,"NUMBER(10,2)"
8,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,LILHD_NDX,생활지수,NUMBER(24)
9,TB_APRTM_RIVER_STRMS_LVSPC,서울시 아파트별 한강 및 지천 생활권 정보,LILHD_NDX_NM,생활지수명,VARCHAR(10)


In [235]:
index["컬럼명(영문)"] = index["컬럼명(영문)"].str.lower()

In [236]:
variables = index.set_index('컬럼명(영문)').to_dict()['컬럼명(한글)'] # 코드북에 있는 한글 변수명 column을 키로 영문 변수명 column을 value로 한 딕셔너리타입 변수 생성
variables

{'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'nmhsh': '세대수',
 'lgdng_addr': '법정동주소',
 'cours_cnt': '코스수',
 'cours_tot_lngth': '코스총길이',
 'lilhd_ndx': '생활지수',
 'lilhd_ndx_nm': '생활지수명',
 'evscr': '평가점수'}

In [237]:
variables = {'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'nmhsh': '세대수',
 'lgdng_addr': '법정동주소',
 'cours_cnt': '지천코스수',
 'cours_tot_lngth': '지천코스총길이',
 'lilhd_ndx': '지천생활지수',
 'lilhd_ndx_nm': '지천생활지수명',
 'evscr': '지천평가점수'}

In [238]:
river_range.rename(columns=variables, inplace=True) # column의 이름 변경

In [239]:
river_range

,시군구코드,법정동코드,단지코드,단지명,세대수,법정동주소,지천코스수,지천코스총길이,지천생활지수,지천생활지수명,지천평가점수
0,11110,1111011500,A11005401,광화문풍림스페이스본 아파트,744,서울특별시 종로구 사직동 9 광화문풍림스페이스본 아파트,0,0.00,5,하위권,0
1,11110,1111011800,A11007001,경희궁의아침3단지,150,서울특별시 종로구 내수동 72 경희궁의아침3단지,1,8179.50,4,중하위권,84
2,11110,1111011800,A11087101,경희궁의아침2단지,90,서울특별시 종로구 내수동 71 경희궁의아침2단지,1,8179.50,4,중하위권,84
3,11110,1111012400,A11014001,로얄팰리스스위트,23,서울특별시 종로구 수송동 85 로얄팰리스스위트,1,8179.50,4,중하위권,84
4,11110,1111013300,A11034001,현대뜨레비앙,291,서울특별시 종로구 익선동 55 현대뜨레비앙,1,8179.50,4,중하위권,84
...,...,...,...,...,...,...,...,...,...,...,...
2638,11740,1174011000,A13410006,강일리버파크4단지,748,서울특별시 강동구 강일동 673 강일리버파크4단지,2,8064.31,4,중하위권,84
2639,11740,1174011000,A13410004,강일리버파크6단지,553,서울특별시 강동구 강일동 684 강일리버파크6단지,2,8064.31,4,중하위권,84
2640,11740,1174011000,A13410010,강일리버파크7단지,731,서울특별시 강동구 강일동 686 강일리버파크7단지,2,8064.31,4,중하위권,84
2641,11740,1174011000,A13410009,강일리버파크5단지,722,서울특별시 강동구 강일동 670 강일리버파크5단지,2,8064.31,4,중하위권,84


In [240]:
# 사용하지 않는 columns 삭제
use_river_range = river_range.drop(columns=["시군구코드", "법정동코드", "단지코드", "세대수"], axis=1)

In [241]:
use_river_range

,단지명,법정동주소,지천코스수,지천코스총길이,지천생활지수,지천생활지수명,지천평가점수
0,광화문풍림스페이스본 아파트,서울특별시 종로구 사직동 9 광화문풍림스페이스본 아파트,0,0.00,5,하위권,0
1,경희궁의아침3단지,서울특별시 종로구 내수동 72 경희궁의아침3단지,1,8179.50,4,중하위권,84
2,경희궁의아침2단지,서울특별시 종로구 내수동 71 경희궁의아침2단지,1,8179.50,4,중하위권,84
3,로얄팰리스스위트,서울특별시 종로구 수송동 85 로얄팰리스스위트,1,8179.50,4,중하위권,84
4,현대뜨레비앙,서울특별시 종로구 익선동 55 현대뜨레비앙,1,8179.50,4,중하위권,84
...,...,...,...,...,...,...,...
2638,강일리버파크4단지,서울특별시 강동구 강일동 673 강일리버파크4단지,2,8064.31,4,중하위권,84
2639,강일리버파크6단지,서울특별시 강동구 강일동 684 강일리버파크6단지,2,8064.31,4,중하위권,84
2640,강일리버파크7단지,서울특별시 강동구 강일동 686 강일리버파크7단지,2,8064.31,4,중하위권,84
2641,강일리버파크5단지,서울특별시 강동구 강일동 670 강일리버파크5단지,2,8064.31,4,중하위권,84


In [242]:
use_river_range["법정동주소"] = use_river_range.apply(remove_substring, axis=1)

In [243]:
use_river_range["법정동주소"] = use_river_range["법정동주소"].str.strip()

In [244]:
use_river_range = use_river_range[~use_river_range["법정동주소"].duplicated()]

In [245]:
use_river_range

,단지명,법정동주소,지천코스수,지천코스총길이,지천생활지수,지천생활지수명,지천평가점수
0,광화문풍림스페이스본 아파트,서울특별시 종로구 사직동 9,0,0.00,5,하위권,0
1,경희궁의아침3단지,서울특별시 종로구 내수동 72,1,8179.50,4,중하위권,84
2,경희궁의아침2단지,서울특별시 종로구 내수동 71,1,8179.50,4,중하위권,84
3,로얄팰리스스위트,서울특별시 종로구 수송동 85,1,8179.50,4,중하위권,84
4,현대뜨레비앙,서울특별시 종로구 익선동 55,1,8179.50,4,중하위권,84
...,...,...,...,...,...,...,...
2638,강일리버파크4단지,서울특별시 강동구 강일동 673,2,8064.31,4,중하위권,84
2639,강일리버파크6단지,서울특별시 강동구 강일동 684,2,8064.31,4,중하위권,84
2640,강일리버파크7단지,서울특별시 강동구 강일동 686,2,8064.31,4,중하위권,84
2641,강일리버파크5단지,서울특별시 강동구 강일동 670,2,8064.31,4,중하위권,84


In [246]:
del use_river_range["단지명"]

In [247]:
new_concat = pd.merge(new_concat, use_river_range, on=["법정동주소"], how='left')

In [248]:
concat.shape

(1128094, 60)

In [249]:
new_concat.shape

(1128094, 72)

### 8. Near the park imformation Merge

In [250]:
# 도시공원 생활권 정보
city_park_path = '../../data/city_park.csv'
index_path = '../../data/city_park_categori.csv'
city_park = pd.read_csv(city_park_path, encoding='cp949')
index = pd.read_csv(index_path, encoding='cp949')

In [251]:
city_park

,sgng_cd,lgdng_cd,hscmp_cd,hscmp_nm,nmhsh,lgdng_addr,park_cnt,park_ttar,lilhd_ndx,lilhd_ndx_nm,evscr
0,11110,1111011500,A11005401,광화문풍림스페이스본 아파트,744,서울특별시 종로구 사직동 9 광화문풍림스페이스본 아파트,12,317184.27,4,중하위권,81
1,11110,1111011800,A11007001,경희궁의아침3단지,150,서울특별시 종로구 내수동 72 경희궁의아침3단지,16,321615.17,4,중하위권,81
2,11110,1111011800,A11087101,경희궁의아침2단지,90,서울특별시 종로구 내수동 71 경희궁의아침2단지,16,322286.17,4,중하위권,81
3,11110,1111012400,A11014001,로얄팰리스스위트,23,서울특별시 종로구 수송동 85 로얄팰리스스위트,15,67828.60,4,중하위권,71
4,11110,1111013300,A11034001,현대뜨레비앙,291,서울특별시 종로구 익선동 55 현대뜨레비앙,8,533213.30,3,중위권,85
...,...,...,...,...,...,...,...,...,...,...,...
2638,11740,1174011000,A13410006,강일리버파크4단지,748,서울특별시 강동구 강일동 673 강일리버파크4단지,7,176616.90,4,중하위권,78
2639,11740,1174011000,A13410004,강일리버파크6단지,553,서울특별시 강동구 강일동 684 강일리버파크6단지,10,309300.10,4,중하위권,81
2640,11740,1174011000,A13410010,강일리버파크7단지,731,서울특별시 강동구 강일동 686 강일리버파크7단지,9,209700.20,4,중하위권,79
2641,11740,1174011000,A13410009,강일리버파크5단지,722,서울특별시 강동구 강일동 670 강일리버파크5단지,7,176616.90,4,중하위권,78


In [252]:
index

,테이블명(영문),테이블명(한글),컬럼명(영문),컬럼명(한글),타입
0,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,SGNG_CD,시군구코드,VARCHAR(5)
1,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,LGDNG_CD,법정동코드,VARCHAR(10)
2,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,HSCMP_CD,단지코드,VARCHAR(9)
3,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,HSCMP_NM,단지명,VARCHAR(50)
4,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,NMHSH,세대수,NUMBER(20)
5,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,LGDNG_ADDR,법정동주소,VARCHAR(100)
6,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,PARK_CNT,공원수,NUMBER(14)
7,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,PARK_TTAR,공원총면적,"NUMBER(13,4)"
8,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,LILHD_NDX,생활지수,NUMBER(24)
9,TB_APRTM_FREST_LVSPC,서울시 아파트별 도시공원 생활권 정보,LILHD_NDX_NM,생활지수명,VARCHAR(10)


In [253]:
index["컬럼명(영문)"] = index["컬럼명(영문)"].str.lower()

In [254]:
variables = index.set_index('컬럼명(영문)').to_dict()['컬럼명(한글)'] # 코드북에 있는 한글 변수명 column을 키로 영문 변수명 column을 value로 한 딕셔너리타입 변수 생성
variables

{'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'nmhsh': '세대수',
 'lgdng_addr': '법정동주소',
 'park_cnt': '공원수',
 'park_ttar': '공원총면적',
 'lilhd_ndx': '생활지수',
 'lilhd_ndx_nm': '생활지수명',
 'evscr': '평가점수'}

In [255]:
variables = {'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'nmhsh': '세대수',
 'lgdng_addr': '법정동주소',
 'park_cnt': '공원수',
 'park_ttar': '공원총면적',
 'lilhd_ndx': '도시공원생활지수',
 'lilhd_ndx_nm': '도시공원생활지수명',
 'evscr': '도시공원평가점수'}

In [256]:
city_park.rename(columns=variables, inplace=True) # column의 이름 변경

In [257]:
use_city_park = city_park.drop(columns=["시군구코드", "법정동코드", "단지코드", "세대수"], axis=1)

In [258]:
use_city_park["법정동주소"] = use_city_park.apply(remove_substring, axis=1)
use_city_park["법정동주소"] = use_city_park["법정동주소"].str.strip()
use_city_park = use_city_park[~use_city_park["법정동주소"].duplicated()]

In [259]:
del use_city_park["단지명"]

In [260]:
new_concat = pd.merge(new_concat, use_city_park, on=["법정동주소"], how='left')

In [261]:
concat.shape

(1128094, 60)

In [263]:
new_concat.shape

(1128094, 77)

### 9. Near the working road imformation Merge

In [264]:
# 도시공원 생활권 정보
walking_path = '../../data/walking.csv'
index_path = '../../data/walking_categori.csv'
walking = pd.read_csv(walking_path, encoding='cp949')
index = pd.read_csv(index_path, encoding='cp949')

In [265]:
walking

,sgng_cd,lgdng_cd,hscmp_cd,hscmp_nm,nmhsh,lgdng_addr,cours_cnt,cours_tot_lngth,lilhd_ndx,lilhd_ndx_nm,evscr
0,11110,1111011500,A11005401,광화문풍림스페이스본 아파트,744,서울특별시 종로구 사직동 9 광화문풍림스페이스본 아파트,9,16916.61,1,상위권,95
1,11110,1111011800,A11007001,경희궁의아침3단지,150,서울특별시 종로구 내수동 72 경희궁의아침3단지,8,15662.61,1,상위권,94
2,11110,1111011800,A11087101,경희궁의아침2단지,90,서울특별시 종로구 내수동 71 경희궁의아침2단지,8,15662.61,1,상위권,94
3,11110,1111012400,A11014001,로얄팰리스스위트,23,서울특별시 종로구 수송동 85 로얄팰리스스위트,6,12605.88,2,중상위권,92
4,11110,1111013300,A11034001,현대뜨레비앙,291,서울특별시 종로구 익선동 55 현대뜨레비앙,4,11385.21,2,중상위권,91
...,...,...,...,...,...,...,...,...,...,...,...
2638,11740,1174011000,A13410006,강일리버파크4단지,748,서울특별시 강동구 강일동 673 강일리버파크4단지,0,0.00,5,하위권,0
2639,11740,1174011000,A13410004,강일리버파크6단지,553,서울특별시 강동구 강일동 684 강일리버파크6단지,2,3236.52,4,중하위권,79
2640,11740,1174011000,A13410010,강일리버파크7단지,731,서울특별시 강동구 강일동 686 강일리버파크7단지,0,0.00,5,하위권,0
2641,11740,1174011000,A13410009,강일리버파크5단지,722,서울특별시 강동구 강일동 670 강일리버파크5단지,0,0.00,5,하위권,0


In [266]:
index

,테이블명(영문),테이블명(한글),컬럼명(영문),컬럼명(한글),타입
0,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,SGNG_CD,시군구코드,VARCHAR(5)
1,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,LGDNG_CD,법정동코드,VARCHAR(10)
2,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,HSCMP_CD,단지코드,VARCHAR(9)
3,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,HSCMP_NM,단지명,VARCHAR(50)
4,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,NMHSH,세대수,NUMBER(20)
5,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,LGDNG_ADDR,법정동주소,VARCHAR(100)
6,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,COURS_CNT,코스수,NUMBER(14)
7,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,COURS_TOT_LNGTH,코스총길이,"NUMBER(10,2)"
8,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,LILHD_NDX,생활지수,NUMBER(24)
9,TB_APRTM_WLK_LVSPC,서울시 아파트별 산책 생활권 정보,LILHD_NDX_NM,생활지수명,VARCHAR(10)


In [268]:
index["컬럼명(영문)"] = index["컬럼명(영문)"].str.lower()

In [269]:
variables = index.set_index('컬럼명(영문)').to_dict()['컬럼명(한글)'] # 코드북에 있는 한글 변수명 column을 키로 영문 변수명 column을 value로 한 딕셔너리타입 변수 생성
variables

{'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'nmhsh': '세대수',
 'lgdng_addr': '법정동주소',
 'cours_cnt': '코스수',
 'cours_tot_lngth': '코스총길이',
 'lilhd_ndx': '생활지수',
 'lilhd_ndx_nm': '생활지수명',
 'evscr': '평가점수'}

In [270]:
variables = {'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'nmhsh': '세대수',
 'lgdng_addr': '법정동주소',
 'cours_cnt': '산책코스수',
 'cours_tot_lngth': '산책코스총길이',
 'lilhd_ndx': '산책생활지수',
 'lilhd_ndx_nm': '산책생활지수명',
 'evscr': '산책평가점수'}

In [271]:
walking.rename(columns=variables, inplace=True) # column의 이름 변경

In [272]:
use_walking = walking.drop(columns=["시군구코드", "법정동코드", "단지코드", "세대수"], axis=1)

In [273]:
use_walking

,단지명,법정동주소,산책코스수,산책코스총길이,산책생활지수,산책생활지수명,산책평가점수
0,광화문풍림스페이스본 아파트,서울특별시 종로구 사직동 9 광화문풍림스페이스본 아파트,9,16916.61,1,상위권,95
1,경희궁의아침3단지,서울특별시 종로구 내수동 72 경희궁의아침3단지,8,15662.61,1,상위권,94
2,경희궁의아침2단지,서울특별시 종로구 내수동 71 경희궁의아침2단지,8,15662.61,1,상위권,94
3,로얄팰리스스위트,서울특별시 종로구 수송동 85 로얄팰리스스위트,6,12605.88,2,중상위권,92
4,현대뜨레비앙,서울특별시 종로구 익선동 55 현대뜨레비앙,4,11385.21,2,중상위권,91
...,...,...,...,...,...,...,...
2638,강일리버파크4단지,서울특별시 강동구 강일동 673 강일리버파크4단지,0,0.00,5,하위권,0
2639,강일리버파크6단지,서울특별시 강동구 강일동 684 강일리버파크6단지,2,3236.52,4,중하위권,79
2640,강일리버파크7단지,서울특별시 강동구 강일동 686 강일리버파크7단지,0,0.00,5,하위권,0
2641,강일리버파크5단지,서울특별시 강동구 강일동 670 강일리버파크5단지,0,0.00,5,하위권,0


In [274]:
use_walking["법정동주소"] = use_walking.apply(remove_substring, axis=1)
use_walking["법정동주소"] = use_walking["법정동주소"].str.strip()
use_walking = use_walking[~use_walking["법정동주소"].duplicated()]

In [275]:
use_walking

,단지명,법정동주소,산책코스수,산책코스총길이,산책생활지수,산책생활지수명,산책평가점수
0,광화문풍림스페이스본 아파트,서울특별시 종로구 사직동 9,9,16916.61,1,상위권,95
1,경희궁의아침3단지,서울특별시 종로구 내수동 72,8,15662.61,1,상위권,94
2,경희궁의아침2단지,서울특별시 종로구 내수동 71,8,15662.61,1,상위권,94
3,로얄팰리스스위트,서울특별시 종로구 수송동 85,6,12605.88,2,중상위권,92
4,현대뜨레비앙,서울특별시 종로구 익선동 55,4,11385.21,2,중상위권,91
...,...,...,...,...,...,...,...
2638,강일리버파크4단지,서울특별시 강동구 강일동 673,0,0.00,5,하위권,0
2639,강일리버파크6단지,서울특별시 강동구 강일동 684,2,3236.52,4,중하위권,79
2640,강일리버파크7단지,서울특별시 강동구 강일동 686,0,0.00,5,하위권,0
2641,강일리버파크5단지,서울특별시 강동구 강일동 670,0,0.00,5,하위권,0


In [276]:
del use_walking["단지명"]

In [277]:
new_concat = pd.merge(new_concat, use_walking, on=["법정동주소"], how='left')

In [278]:
concat.shape

(1128094, 60)

In [281]:
new_concat.shape

(1128094, 82)

### 10. Near the ecological park imformation Merge

In [282]:
# 생태공원 정보
ecology_path = '../../data/ecology.csv'
index_path = '../../data/ecology_categori.csv'
ecology = pd.read_csv(ecology_path, encoding='cp949')
index = pd.read_csv(index_path, encoding='cp949')

In [283]:
index["컬럼명(영문)"] = index["컬럼명(영문)"].str.lower()

In [284]:
variables = index.set_index('컬럼명(영문)').to_dict()['컬럼명(한글)'] # 코드북에 있는 한글 변수명 column을 키로 영문 변수명 column을 value로 한 딕셔너리타입 변수 생성
variables

{'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'nmhsh': '세대수',
 'lgdng_addr': '법정동주소',
 'cours_cnt': '코스수',
 'cours_tot_lngth': '코스총길이',
 'lilhd_ndx': '생활지수',
 'lilhd_ndx_nm': '생활지수명',
 'evscr': '평가점수'}

In [285]:
variables = {'sgng_cd': '시군구코드',
 'lgdng_cd': '법정동코드',
 'hscmp_cd': '단지코드',
 'hscmp_nm': '단지명',
 'nmhsh': '세대수',
 'lgdng_addr': '법정동주소',
 'cours_cnt': '생태공원코스수',
 'cours_tot_lngth': '생태공원코스총길이',
 'lilhd_ndx': '생태공원생활지수',
 'lilhd_ndx_nm': '생태공원생활지수명',
 'evscr': '생태공원평가점수'}

In [286]:
ecology.rename(columns=variables, inplace=True) # column의 이름 변경

In [287]:
use_ecology = ecology.drop(columns=["시군구코드", "법정동코드", "단지코드", "세대수"], axis=1)

In [288]:
use_ecology["법정동주소"] = use_ecology.apply(remove_substring, axis=1)
use_ecology["법정동주소"] = use_ecology["법정동주소"].str.strip()
use_ecology = use_ecology[~use_ecology["법정동주소"].duplicated()]

In [289]:
del use_ecology["단지명"]

In [291]:
new_concat = pd.merge(new_concat, use_ecology, on=["법정동주소"], how='left')

In [292]:
new_concat.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target', 'is_test', '계약년', '계약월', '구', '동',
       '국내총생산(명목GDP)', '경제성장률(실질GDP성장률)', '기준금리', '학군', '전세가율', '재개발개수',
       '법정동주소', '숲세권생활지수', '숲세권생활지수명', '숲세권평가점수', '지천코스수', '지천코스총길이', '지천생활지수',
       '지천생활지수명', '지천평가점수', '공원수', '공원총면적', '도시공원생활지수', '도시공원생활지수명',
       '도시공원평가점수', '산책코스수', '산책코스총길이', '산책생활지수', '산책생활지수명', '산책평가점수',
       '생태공원코스수', '생태공원코스총길이', '생태공원생활지수'

In [293]:
concat.shape

(1128094, 60)

In [294]:
new_concat.shape

(1128094, 87)

## 4. Data Save

In [ ]:
save_train = new_concat[new_concat["is_test"]==0]
del save_train["is_test"]
save_train.to_csv('train_merge_0122_01(cp949)_final.csv', index=False, encoding='cp949')
save_train.to_csv('train_merge_0122_01(UTF-8)_final.csv', index=False, encoding='UTF-8')

In [ ]:
save_test = new_concat[new_concat["is_test"]==1]
del save_test["is_test"]
save_test.to_csv('test_merge_0122_01(cp949)_final.csv', index=False, encoding='cp949')
save_test.to_csv('test_merge_0122_01(UTF-8)_final.csv', index=False, encoding='UTF-8')